In [ ]:
# https://claude.ai/chat/6bee8cdc-4575-45a2-aa49-0eb8eac09d54
# https://tensornetwork.org/mps/algorithms/dmrg/
# https://tensornetwork.org/mps/
# https://tensornetwork.org/mpo/

import copy
import torch
import numpy as np
from typing import List, Tuple, Optional, Union, Callable


class MPS:
    """Matrix Product State (MPS) implementation.
    矩阵乘积态是一种将高维张量分解为一维链状三指标张量乘积的方法，
    用于高效表示量子多体系统的波函数。
    """

    def __init__(self,
                 physical_dims: Union[List[int], int],
                 bond_dims: Union[List[int], int],
                 num_sites: Optional[int] = None,
                 random_init: bool = True,
                 device: Optional[torch.device] = None,
                 dtype: torch.dtype = torch.float32,
                 max_bond_dim: int = 1000):
        """Initialize the MPS with random tensors.
        :param physical_dims: 每个位点的物理维度(local Hilbert space dimension)，可以是单一整数或列表
        :param bond_dims: 键维度，可以是单一整数或列表
        :param num_sites: 链的长度，如果physical_dims是列表则可以为None推断出来
        :param random_init: If True, initialize with random tensors
        :param device: Device to store the tensors (e.g., "cpu" or "cuda")
        :param dtype: Data type of the tensors (e.g., torch.float32)
        :param max_bond_dim: 最大键维度
        """
        self.device = device if device is not None else torch.device("cpu")
        self.dtype = dtype
        self.max_bond_dim = max_bond_dim

        # 处理物理维度
        if isinstance(physical_dims, int):
            assert num_sites is not None, "如果physical_dims是单一整数，必须指定num_sites"
            assert num_sites > 0, "num_sites必须大于0"
            assert physical_dims > 0, "physical_dims必须大于0"
            self.physical_dims = [physical_dims] * num_sites
            self.length = num_sites
        else:
            assert num_sites is None or len(physical_dims) == num_sites, "physical_dims的长度必须与num_sites相同"
            assert all(d > 0 for d in physical_dims), "physical_dims的元素必须大于0"
            self.physical_dims = physical_dims
            self.length = len(physical_dims)

        # 处理键维度
        if isinstance(bond_dims, int):
            assert bond_dims > 0, "bond_dims必须大于0"
            self.bond_dims = [bond_dims] * (self.length - 1)  # 键维度比位点数少1
        else:
            assert len(bond_dims) == self.length - 1, "bond_dims的长度必须与num_sites-1相同"
            assert all(d > 0 for d in bond_dims), "bond_dims的元素必须大于0"
            self.bond_dims = bond_dims

        # 初始化MPS张量列表
        self.tensors = []
        if random_init:
            self._random_init()

        # 正交中心位置，初始化为None表示尚未正交化
        self.center = None

    def _random_init(self):
        self.tensors = []
        # 头部张量：(physical_dim, bond_dim)
        head_tensor = torch.randn(self.physical_dims[0], self.bond_dims[0],
                                  device=self.device, dtype=self.dtype)
        head_tensor /= torch.norm(head_tensor)  # 归一化
        self.tensors.append(head_tensor)

        # 中间张量：(physical_dim, bond_dim_left, bond_dim_right)
        for i in range(1, self.length - 1):
            mid_tensor = torch.randn(self.physical_dims[i], self.bond_dims[i - 1], self.bond_dims[i],
                                     device=self.device, dtype=self.dtype)
            mid_tensor /= torch.norm(mid_tensor)  # 归一化
            self.tensors.append(mid_tensor)

        # 尾部张量：(physical_dim, bond_dim)
        tail_tensor = torch.randn(self.physical_dims[-1], self.bond_dims[-1],
                                  device=self.device, dtype=self.dtype)
        tail_tensor /= torch.norm(tail_tensor)  # 归一化
        self.tensors.append(tail_tensor)

    def _truncate_svd(self, u: torch.Tensor, s: torch.Tensor, vh: torch.Tensor) -> Tuple[
        torch.Tensor, torch.Tensor, torch.Tensor]:
        """对奇异值进行截断，保留最大的max_bond_dim个奇异值"""
        if s.size(0) > self.max_bond_dim:
            u = u[:, :self.max_bond_dim]
            s = s[:self.max_bond_dim]
            vh = vh[:self.max_bond_dim, :]
        return u, s, vh

    def right_canonicalize(self):
        """将MPS正交化为右正则形式。在右正则形式中，除了头部张量外都满足正则化条件"""
        if self.length == 1:
            self.center = 0
            return
        for i in range(self.length - 1, 0, -1):  # 从最后一个张量开始正交化
            self._right_canonicalize_step(i)
        self.center = 0

    def _right_canonicalize_step(self, site: int):
        """Perform a single step of right canonicalization.
        :param site: 要正交化的位点索引
        """
        # 获取当前张量
        tensor = self.tensors[site]
        shape = tensor.shape

        if site == self.length - 1:  # 尾部位点：(physical_dim * bond_dim, 1)
            matrix = tensor.reshape(-1, 1)
            q, r = torch.linalg.qr(matrix)
            self.tensors[site] = q.reshape(shape[0], shape[1], 1)
            if site > 0: # 将R矩阵传递给左侧张量
                self.tensors[site - 1] = torch.tensordot(self.tensors[site - 1], r, dims=([2], [1]))
        else:  # 中间位点：(physical_dim * bond_dim_left, bond_dim_right)
            matrix = self.tensors(-1, shape[2])


In [2]:
torch.swapdims()

4
3
2
1


In [ ]:
import numpy as np
from typing import List, Tuple, Optional, Union
import copy
from refactored_mps import MPS


class MPO:
    """
    矩阵乘积算符 (Matrix Product Operator)

    MPO是一种将高维张量（具有N个协变和N个逆变指标）分解为一维链状四指标张量乘积的方法，
    用于高效表示量子多体系统的算符。

    属性:
        physical_dims: 每个位点的物理维度列表
        bond_dims: 键维度列表
        length: 系统长度
        tensors: MPO张量列表
        device: PyTorch设备
        dtype: 数据类型
    """

    def __init__(self,
                 physical_dims: Union[List[int], int],
                 bond_dims: Union[List[int], int],
                 length: Optional[int] = None,
                 random_init: bool = True,
                 device: Optional[torch.device] = None,
                 dtype: torch.dtype = torch.float32):
        """
        初始化一个MPO

        参数:
            physical_dims: 每个位点的物理维度，可以是单一整数或列表
            bond_dims: 键维度，可以是单一整数或列表
            length: 链的长度，如果physical_dims是列表则可以为None
            random_init: 是否随机初始化张量
            device: PyTorch设备
            dtype: 数据类型
        """
        self.device = device if device is not None else torch.device('cpu')
        self.dtype = dtype

        # 处理物理维度
        if isinstance(physical_dims, int):
            assert length is not None, "如果physical_dims是单一整数，必须指定length"
            self.physical_dims = [physical_dims] * length
            self.length = length
        else:
            self.physical_dims = physical_dims
            self.length = len(physical_dims)

        # 处理键维度
        if isinstance(bond_dims, int):
            # 键维度比位点数少1
            self.bond_dims = [bond_dims] * (self.length - 1)
        else:
            assert len(bond_dims) == self.length - 1, "键维度数量必须是位点数量-1"
            self.bond_dims = bond_dims

        # 初始化MPO张量列表，每个张量有4个指标：
        # 1. 物理输入指标 (共变)
        # 2. 物理输出指标 (逆变)
        # 3. 左键指标
        # 4. 右键指标
        self.tensors = []

        if random_init:
            self._random_init()

    def _random_init(self):
        """随机初始化MPO张量"""
        self.tensors = []

        # 第一个张量：(physical_in, physical_out, 1, bond_dim)
        # 第一个位点没有左键指标，所以用1代替
        first_tensor = torch.randn(self.physical_dims[0], self.physical_dims[0],
                                   1, self.bond_dims[0],
                                   device=self.device, dtype=self.dtype)
        self.tensors.append(first_tensor)

        # 中间张量：(physical_in, physical_out, bond_dim_left, bond_dim_right)
        for i in range(1, self.length - 1):
            tensor = torch.randn(self.physical_dims[i], self.physical_dims[i],
                                 self.bond_dims[i - 1], self.bond_dims[i],
                                 device=self.device, dtype=self.dtype)
            self.tensors.append(tensor)

        # 最后一个张量：(physical_in, physical_out, bond_dim, 1)
        # 最后一个位点没有右键指标，所以用1代替
        last_tensor = torch.randn(self.physical_dims[-1], self.physical_dims[-1],
                                  self.bond_dims[-1], 1,
                                  device=self.device, dtype=self.dtype)
        self.tensors.append(last_tensor)

    def apply(self, mps: MPS) -> MPS:
        """
        将MPO应用于MPS，返回新的MPS

        参数:
            mps: 输入的MPS

        返回:
            MPO作用后的新MPS
        """
        assert self.length == mps.length, "MPO和MPS长度必须相同"
        assert all(d_mpo == d_mps for d_mpo, d_mps in zip(self.physical_dims, mps.physical_dims)), "物理维度必须匹配"

        # 创建新的MPS
        # 新MPS的键维度是MPO和MPS键维度的乘积
        new_bond_dims = []
        for i in range(self.length - 1):
            new_bond_dims.append(self.bond_dims[i] * mps.get_bond_dimensions()[i])

        result = MPS(self.physical_dims, new_bond_dims, random_init=False,
                     device=self.device, dtype=self.dtype)

        # 对每个位点应用MPO
        for i in range(self.length):
            if i == 0:
                self._apply_first_site(i, mps, result)
            elif i == self.length - 1:
                self._apply_last_site(i, mps, result)
            else:
                self._apply_middle_site(i, mps, result)

        return result

    def _apply_first_site(self, site: int, mps: MPS, result: MPS):
        """
        对第一个位点应用MPO

        参数:
            site: 位点索引
            mps: 输入MPS
            result: 结果MPS
        """
        # 第一个位点特殊处理
        # MPO张量: (p_in, p_out, 1, bond_mpo)
        # MPS张量: (p, bond_mps)
        mpo_tensor = self.tensors[site]
        mps_tensor = mps.tensors[site]

        # 收缩物理指标: (p_out, 1, bond_mpo, bond_mps)
        result_tensor = torch.einsum('pqab,pb->qabp', mpo_tensor, mps_tensor)

        # 重塑为正确的形状: (p_out, bond_mpo * bond_mps)
        shape = result_tensor.shape
        result.tensors.append(result_tensor.reshape(shape[0], shape[1] * shape[2] * shape[3]))

    def _apply_last_site(self, site: int, mps: MPS, result: MPS):
        """
        对最后一个位点应用MPO

        参数:
            site: 位点索引
            mps: 输入MPS
            result: 结果MPS
        """
        # 最后一个位点特殊处理
        # MPO张量: (p_in, p_out, bond_mpo, 1)
        # MPS张量: (p, bond_mps)
        mpo_tensor = self.tensors[site]
        mps_tensor = mps.tensors[site]

        # 收缩物理指标: (p_out, bond_mpo, 1, bond_mps)
        result_tensor = torch.einsum('pqab,pb->qabp', mpo_tensor, mps_tensor)

        # 重塑为正确的形状: (p_out, bond_mpo * bond_mps)
        shape = result_tensor.shape
        result.tensors.append(result_tensor.reshape(shape[0], shape[1] * shape[2] * shape[3]))

    def _apply_middle_site(self, site: int, mps: MPS, result: MPS):
        """
        对中间位点应用MPO

        参数:
            site: 位点索引
            mps: 输入MPS
            result: 结果MPS
        """
        # 中间位点
        # MPO张量: (p_in, p_out, bond_mpo_left, bond_mpo_right)
        # MPS张量: (p, bond_mps_left, bond_mps_right)
        mpo_tensor = self.tensors[site]
        mps_tensor = mps.tensors[site]

        # 收缩物理指标: (p_out, bond_mpo_left, bond_mpo_right, bond_mps_left, bond_mps_right)
        result_tensor = torch.einsum('pqab,pcd->qabcd', mpo_tensor, mps_tensor)

        # 重塑为正确的形状: (p_out, bond_mpo_left * bond_mps_left, bond_mpo_right * bond_mps_right)
        shape = result_tensor.shape
        result.tensors.append(result_tensor.reshape(shape[0],
                                                    shape[1] * shape[3],
                                                    shape[2] * shape[4]))

    def contract(self) -> torch.Tensor:
        """
        将整个MPO收缩为完整的矩阵/算符

        返回:
            完整的算符表示，形状为(D, D)，其中D是整个物理空间的维度
        """
        # 计算总物理维度
        total_dim = 1
        for d in self.physical_dims:
            total_dim *= d

        # 从第一个张量开始
        result = self.tensors[0]

        # 逐一收缩所有张量
        for i in range(1, self.length):
            # 收缩相邻张量的键指标
            result = torch.tensordot(result, self.tensors[i], dims=([3], [2]))

        # 重塑结果为矩阵形式
        result = self._reshape_to_matrix(result, total_dim)

        return result

    def _reshape_to_matrix(self, tensor: torch.Tensor, total_dim: int) -> torch.Tensor:
        """
        将张量重塑为矩阵形式

        参数:
            tensor: 输入张量
            total_dim: 总物理维度

        返回:
            矩阵形式的张量
        """
        # 将所有物理输入指标合并为一个指标，所有物理输出指标合并为另一个指标
        perm = []
        # 所有物理输入指标在前
        for i in range(self.length):
            perm.append(i * 2)
        # 所有物理输出指标在后
        for i in range(self.length):
            perm.append(i * 2 + 1)

        result = tensor.permute(perm)

        # 重塑为矩阵
        result = result.reshape(total_dim, total_dim)

        return result

    @classmethod
    def identity(cls, physical_dims, length=None, device=None, dtype=torch.float32):
        """
        创建一个表示恒等算符的MPO

        参数:
            physical_dims: 物理维度，可以是整数或列表
            length: 系统长度（如果physical_dims是整数）
            device: PyTorch设备
            dtype: 数据类型

        返回:
            恒等算符MPO
        """
        if isinstance(physical_dims, int):
            assert length is not None, "如果physical_dims是整数，必须指定length"
            physical_dims = [physical_dims] * length
        else:
            length = len(physical_dims)

        # 创建恒等MPO，键维度为1
        mpo = cls(physical_dims, 1, length, random_init=False, device=device, dtype=dtype)

        # 设置每个位点为局部恒等算符
        for i in range(length):
            cls._set_identity_site(mpo, i, physical_dims[i], device, dtype)

        return mpo

    @staticmethod
    def _set_identity_site(mpo: 'MPO', site: int, physical_dim: int, device, dtype):
        """
        设置指定位点为恒等算符

        参数:
            mpo: MPO对象
            site: 位点索引
            physical_dim: 物理维度
            device: 设备
            dtype: 数据类型
        """
        tensor = torch.zeros((physical_dim, physical_dim, 1, 1),
                             device=device, dtype=dtype)
        for p in range(physical_dim):
            tensor[p, p, 0, 0] = 1.0
        mpo.tensors.append(tensor)

    @classmethod
    def local_operator(cls, physical_dims, site, operator, length=None, device=None, dtype=torch.float32):
        """
        创建一个表示局部算符的MPO

        参数:
            physical_dims: 物理维度，可以是整数或列表
            site: 应用算符的位点
            operator: 局部算符，形状为(physical_dims[site], physical_dims[site])
            length: 系统长度（如果physical_dims是整数）
            device: PyTorch设备
            dtype: 数据类型

        返回:
            局部算符MPO
        """
        if isinstance(physical_dims, int):
            assert length is not None, "如果physical_dims是整数，必须指定length"
            physical_dims = [physical_dims] * length
        else:
            length = len(physical_dims)

        assert 0 <= site < length, f"位点索引{site}超出范围"

        # 创建MPO，键维度为1
        mpo = cls(physical_dims, 1, length, random_init=False, device=device, dtype=dtype)

        # 设置每个位点
        for i in range(length):
            if i == site:
                # 在指定位点应用算符
                cls._set_operator_site(mpo, i, physical_dims[i], operator, device, dtype)
            else:
                # 其他位点应用恒等算符
                cls._set_identity_site(mpo, i, physical_dims[i], device, dtype)

        return mpo

    @staticmethod
    def _set_operator_site(mpo: 'MPO', site: int, physical_dim: int, operator: torch.Tensor, device, dtype):
        """
        设置指定位点为指定算符

        参数:
            mpo: MPO对象
            site: 位点索引
            physical_dim: 物理维度
            operator: 算符
            device: 设备
            dtype: 数据类型
        """
        tensor = torch.zeros((physical_dim, physical_dim, 1, 1),
                             device=device, dtype=dtype)
        tensor[:, :, 0, 0] = operator
        mpo.tensors.append(tensor)